### Imports

In [520]:
from random import choice
from random import random

### Variable definitions

In [521]:



patients = [
    {'Menopause ge40': True, 'Inv-nodes 3-5': True, 'Deg-malign 3': True},
    {'Menopause lt40': True, 'Inv-nodes 0-2': True, 'Deg-malign 3': True},
    {'Menopause ge40': True, 'Inv-nodes 6-8': True, 'Deg-malign 3': True},
    {'Menopause ge40': True, 'Inv-nodes 0-2': True, 'Deg-malign 2': True},
    {'Menopause premeno': True, 'Inv-nodes 0-2': True, 'Deg-malign 3': True},
    {'Menopause premeno':True , 'Inv-nodes 0-2': True, 'Deg-malign 1': True}
]

Rules = [{'Deg-malign 3', 'NOT Menopause lt40'},
         {'Deg-malign 3'},
         {'NOT Inv-nodes 0-2'}]


### Evaluation of features based on rules

In [522]:
def evaluate_condition(observation, condition):
    truth_value_of_condition = True
    for feature in observation:
        if feature in condition and observation[feature] == False:
            truth_value_of_condition = False
            break
        if 'NOT ' + feature in condition and observation[feature] == True:
            truth_value_of_condition = False
            break
    return truth_value_of_condition


### Memory system

In [523]:
class Memory:
    def __init__(self, forget_value, memorize_value, memory):
        self.memory = memory
        self.forget_value = forget_value
        self.memorize_value = memorize_value
        self.memoryAmount = 100
        self.memoryLine = (self.memoryAmount / 2) + 1

    def get_memory(self):
        return self.memory

    def get_literals(self):
        return list(self.memory.keys())

    def get_condition(self):
        condition = []
        for literal in self.memory:
            if self.memory[literal] >= self.memoryLine:
                condition.append(literal)
        return condition

    def memorize(self, literal):
        if random() <= self.memorize_value and self.memory[literal] < self.memoryAmount:
            self.memory[literal] += 1

    def forget(self, literal):
        if random() <= self.forget_value and self.memory[literal] > 1:
            self.memory[literal] -= 1

    def memorize_always(self, literal):
        if  self.memory[literal] < self.memoryAmount:
            self.memory[literal] += 1

### Type i and ii feedback

In [524]:
def type_i_feedback(observation, memory):
    remaining_literals = memory.get_literals()
    #print(remaining_literals)
    if evaluate_condition(observation, memory.get_condition()):
        for feature in observation:
            #print('Feature: ' + feature)
            if observation[feature]:
                memory.memorize(feature)
                remaining_literals.remove(feature)
            elif not observation[feature]:
                memory.memorize('NOT ' + feature)
                remaining_literals.remove('NOT ' + feature)
    for literal in remaining_literals:
        #print('Remaining Literal:' + literal)
        memory.forget(literal)
        
def type_ii_feedback(observation, memory):
    if evaluate_condition(observation, memory.get_condition()):
        for feature in observation:
            if not observation[feature]:
                memory.memorize_always(feature)
            elif observation[feature]:
                memory.memorize_always('NOT ' + feature)

### Boolinization of multi-valued properties 

In [525]:
def GenerateRules():
    StartingMemory = 50
    RecurrenceConditions = {}
    
    # Design-malign rules
    for i in range(3):
        RecurrenceConditions[f'Deg-malign {str(i+1)}'] = StartingMemory
    for i in range(3):
        RecurrenceConditions[f'NOT Deg-malign {str(i+1)}'] = StartingMemory

    # Menopause
    Menopause = ['ge40', 'lt40', 'premeno']
    for menop in Menopause:
        RecurrenceConditions[f'Menopause {menop}'] = StartingMemory
    for menop in Menopause:
        RecurrenceConditions[f'NOT Menopause {menop}'] = StartingMemory

    # Inv-nodes
    InvNodes = ['0-2', '3-5', '6-8']
    for invnode in InvNodes:
        RecurrenceConditions[f'Inv-nodes {invnode}'] = StartingMemory
    for invnode in InvNodes:
        RecurrenceConditions[f'NOT Inv-nodes {invnode}'] = StartingMemory
    
    return RecurrenceConditions

### Classifying the patients and seperating them

In [526]:
ReccurentPatients = []
NonReccurentPatients = []

def ManuallyClassify(observation, rules):
    votes = 0
    for feature in observation:
        for rule in rules:
            if feature in rule and observation[feature] == True:
                votes += 1
            if 'NOT ' + feature in rule and observation[feature] == True:
                votes -= 1
    if votes > 0:
        ReccurentPatients.append(observation)
        return "Recurrence"
    if votes <= 0:
        NonReccurentPatients.append(observation)
        return "Non-Recurrence"

for patient in patients:
    print(ManuallyClassify(patient, Rules))

print('##############################   RECURRENT PATIENTS    ############################################')    
print(ReccurentPatients)
print('#############################   NON-RECURRENT PATIENTS   ##########################################')    
print(NonReccurentPatients)

Recurrence
Non-Recurrence
Recurrence
Non-Recurrence
Recurrence
Non-Recurrence
##############################   RECURRENT PATIENTS    ############################################
[{'Menopause ge40': True, 'Inv-nodes 3-5': True, 'Deg-malign 3': True}, {'Menopause ge40': True, 'Inv-nodes 6-8': True, 'Deg-malign 3': True}, {'Menopause premeno': True, 'Inv-nodes 0-2': True, 'Deg-malign 3': True}]
#############################   NON-RECURRENT PATIENTS   ##########################################
[{'Menopause lt40': True, 'Inv-nodes 0-2': True, 'Deg-malign 3': True}, {'Menopause ge40': True, 'Inv-nodes 0-2': True, 'Deg-malign 2': True}, {'Menopause premeno': True, 'Inv-nodes 0-2': True, 'Deg-malign 1': True}]


### Learning recurrence rule (forget: 0.8, memorize: 0.2)

In [591]:
RecurrenceRule = Memory(0.8,0.2,GenerateRules())
for i in range(10000):
    observation_id = choice([0,1,2])
    patient = choice([0,1])
    if patient == 1:
        type_i_feedback(ReccurentPatients[observation_id], RecurrenceRule)
    else:
        type_ii_feedback(NonReccurentPatients[observation_id], RecurrenceRule)

print('#############################   GET MEMORY   ##########################################')     
print(RecurrenceRule.get_memory())
print('############################  GET CONDITION  ##########################################')
print(RecurrenceRule.get_condition())

#############################   GET MEMORY   ##########################################
{'Deg-malign 1': 1, 'Deg-malign 2': 1, 'Deg-malign 3': 100, 'NOT Deg-malign 1': 1, 'NOT Deg-malign 2': 1, 'NOT Deg-malign 3': 2, 'Menopause ge40': 3, 'Menopause lt40': 1, 'Menopause premeno': 1, 'NOT Menopause ge40': 1, 'NOT Menopause lt40': 1, 'NOT Menopause premeno': 1, 'Inv-nodes 0-2': 1, 'Inv-nodes 3-5': 1, 'Inv-nodes 6-8': 1, 'NOT Inv-nodes 0-2': 49, 'NOT Inv-nodes 3-5': 1, 'NOT Inv-nodes 6-8': 1}
############################  GET CONDITION  ##########################################
['Deg-malign 3']


### Learning non-recurrence rule (forget: 0.8, memorize: 0.2)

In [590]:
NonRecurrenceRule = Memory(0.8, 0.2, GenerateRules())
for i in range(10000):
    observation_id = choice([0, 1, 2])
    patient = choice([0, 1])
    if patient == 1:
        type_i_feedback(NonReccurentPatients[observation_id], NonRecurrenceRule)
    else:
        type_ii_feedback(ReccurentPatients[observation_id], NonRecurrenceRule)

print('#############################   GET MEMORY   ##########################################')
print(NonRecurrenceRule.get_memory())
print('############################  GET CONDITION  ##########################################')
print(NonRecurrenceRule.get_condition())

#############################   GET MEMORY   ##########################################
{'Deg-malign 1': 1, 'Deg-malign 2': 2, 'Deg-malign 3': 1, 'NOT Deg-malign 1': 1, 'NOT Deg-malign 2': 1, 'NOT Deg-malign 3': 50, 'Menopause ge40': 2, 'Menopause lt40': 1, 'Menopause premeno': 1, 'NOT Menopause ge40': 2, 'NOT Menopause lt40': 1, 'NOT Menopause premeno': 1, 'Inv-nodes 0-2': 99, 'Inv-nodes 3-5': 1, 'Inv-nodes 6-8': 1, 'NOT Inv-nodes 0-2': 1, 'NOT Inv-nodes 3-5': 1, 'NOT Inv-nodes 6-8': 2}
############################  GET CONDITION  ##########################################
['Inv-nodes 0-2']


### Classify patients  

In [596]:
def classify(observation, RecurrenceRules, NonRecurrenceRules):
    vote_sum = 0
    for RecurrenceRule in RecurrenceRules:
        if evaluate_condition(observation, RecurrenceRule.get_condition()):
            vote_sum += 1
    for NonRecurrenceRule in NonRecurrenceRules:
        if evaluate_condition(observation, NonRecurrenceRule.get_condition()):
            vote_sum -= 1
    if vote_sum > 0:
        return "Recurrence"
    if vote_sum <= 0:
        return "Non-Recurrence"
    
for patient in patients:
    print(classify(patient, [RecurrenceRule], [NonRecurrenceRule]))

Non-Recurrence
Non-Recurrence
Non-Recurrence
Non-Recurrence
Non-Recurrence
Non-Recurrence


### Task 7

In [694]:
RecurrenceRule = Memory(0.5,0.5,GenerateRules())
for i in range(10000):
    observation_id = choice([0,1,2])
    patient = choice([0,1])
    if patient == 1:
        type_i_feedback(ReccurentPatients[observation_id], RecurrenceRule)
    else:
        type_ii_feedback(NonReccurentPatients[observation_id], RecurrenceRule)

print('#############################   GET MEMORY   ##########################################')     
print(RecurrenceRule.get_memory())
print('############################  GET CONDITION  ##########################################')
print(RecurrenceRule.get_condition())
print()

NonRecurrenceRule = Memory(0.5, 0.5, GenerateRules())
for i in range(10000):
    observation_id = choice([0, 1, 2])
    patient = choice([0, 1])
    if patient == 1:
        type_i_feedback(NonReccurentPatients[observation_id], NonRecurrenceRule)
    else:
        type_ii_feedback(ReccurentPatients[observation_id], NonRecurrenceRule)

print('#############################   GET MEMORY   ##########################################')
print(NonRecurrenceRule.get_memory())
print('############################  GET CONDITION  ##########################################')
print(NonRecurrenceRule.get_condition())
print()

def classify(observation, RecurrenceRules, NonRecurrenceRules):
    vote_sum = 0
    for RecurrenceRule in RecurrenceRules:
        if evaluate_condition(observation, RecurrenceRule.get_condition()):
            vote_sum += 1
    for NonRecurrenceRule in NonRecurrenceRules:
        if evaluate_condition(observation, NonRecurrenceRule.get_condition()):
            vote_sum -= 1
    if vote_sum > 0:
        return "Recurrence"
    if vote_sum <= 0:
        return "Non-Recurrence"

print('################ Classification run ##################')
for patient in patients:
    print(classify(patient, [RecurrenceRule], [NonRecurrenceRule]))

#############################   GET MEMORY   ##########################################
{'Deg-malign 1': 1, 'Deg-malign 2': 1, 'Deg-malign 3': 100, 'NOT Deg-malign 1': 1, 'NOT Deg-malign 2': 4, 'NOT Deg-malign 3': 1, 'Menopause ge40': 98, 'Menopause lt40': 1, 'Menopause premeno': 2, 'NOT Menopause ge40': 3, 'NOT Menopause lt40': 1, 'NOT Menopause premeno': 1, 'Inv-nodes 0-2': 1, 'Inv-nodes 3-5': 3, 'Inv-nodes 6-8': 1, 'NOT Inv-nodes 0-2': 51, 'NOT Inv-nodes 3-5': 1, 'NOT Inv-nodes 6-8': 1}
############################  GET CONDITION  ##########################################
['Deg-malign 3', 'Menopause ge40', 'NOT Inv-nodes 0-2']

#############################   GET MEMORY   ##########################################
{'Deg-malign 1': 3, 'Deg-malign 2': 1, 'Deg-malign 3': 3, 'NOT Deg-malign 1': 1, 'NOT Deg-malign 2': 1, 'NOT Deg-malign 3': 51, 'Menopause ge40': 1, 'Menopause lt40': 1, 'Menopause premeno': 2, 'NOT Menopause ge40': 2, 'NOT Menopause lt40': 1, 'NOT Menopause premeno': 3, 

### Task 8

In [721]:
RecurrenceRule = Memory(0.2,0.8,GenerateRules())
for i in range(10000):
    observation_id = choice([0,1,2])
    patient = choice([0,1])
    if patient == 1:
        type_i_feedback(ReccurentPatients[observation_id], RecurrenceRule)
    else:
        type_ii_feedback(NonReccurentPatients[observation_id], RecurrenceRule)

print('#############################   GET MEMORY   ##########################################')     
print(RecurrenceRule.get_memory())
print('############################  GET CONDITION  ##########################################')
print(RecurrenceRule.get_condition())
print()

NonRecurrenceRule = Memory(0.2, 0.8, GenerateRules())
for i in range(10000):
    observation_id = choice([0, 1, 2])
    patient = choice([0, 1])
    if patient == 1:
        type_i_feedback(NonReccurentPatients[observation_id], NonRecurrenceRule)
    else:
        type_ii_feedback(ReccurentPatients[observation_id], NonRecurrenceRule)

print('#############################   GET MEMORY   ##########################################')
print(NonRecurrenceRule.get_memory())
print('############################  GET CONDITION  ##########################################')
print(NonRecurrenceRule.get_condition())
print()

def classify(observation, RecurrenceRules, NonRecurrenceRules):
    vote_sum = 0
    for RecurrenceRule in RecurrenceRules:
        if evaluate_condition(observation, RecurrenceRule.get_condition()):
            vote_sum += 1
    for NonRecurrenceRule in NonRecurrenceRules:
        if evaluate_condition(observation, NonRecurrenceRule.get_condition()):
            vote_sum -= 1
    if vote_sum > 0:
        return "Recurrence"
    if vote_sum <= 0:
        return "Non-Recurrence"

print('################ Classification run ##################')
for patient in patients:
    print(classify(patient, [RecurrenceRule], [NonRecurrenceRule]))

#############################   GET MEMORY   ##########################################
{'Deg-malign 1': 1, 'Deg-malign 2': 1, 'Deg-malign 3': 100, 'NOT Deg-malign 1': 5, 'NOT Deg-malign 2': 1, 'NOT Deg-malign 3': 2, 'Menopause ge40': 100, 'Menopause lt40': 1, 'Menopause premeno': 4, 'NOT Menopause ge40': 1, 'NOT Menopause lt40': 1, 'NOT Menopause premeno': 1, 'Inv-nodes 0-2': 4, 'Inv-nodes 3-5': 100, 'Inv-nodes 6-8': 98, 'NOT Inv-nodes 0-2': 51, 'NOT Inv-nodes 3-5': 1, 'NOT Inv-nodes 6-8': 1}
############################  GET CONDITION  ##########################################
['Deg-malign 3', 'Menopause ge40', 'Inv-nodes 3-5', 'Inv-nodes 6-8', 'NOT Inv-nodes 0-2']

#############################   GET MEMORY   ##########################################
{'Deg-malign 1': 100, 'Deg-malign 2': 100, 'Deg-malign 3': 1, 'NOT Deg-malign 1': 1, 'NOT Deg-malign 2': 1, 'NOT Deg-malign 3': 51, 'Menopause ge40': 97, 'Menopause lt40': 1, 'Menopause premeno': 100, 'NOT Menopause ge40': 1, 'NOT Men